In [94]:
import random
import re
import pandas as pd
import os

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

def lm_emulator(prompt):
    ### DIES IST KEINE PROMT SONDERN EIN TEST
    print(prompt)
    test_text = f"""
        Ich werde die bereitgestellten Daten analysieren und eine Einschätzung zur Wirkung der Impfkampagne auf die Infektionsrate in NRW im Januar 2022 vornehmen.

        1. Analyse der aktuellen Situation:
        Infektionsrate: 0.5
        Fallzahlen: 1000
        Demografische Impftrends: Frauen werden häufiger geimpft als Männer. Menschen mit Hypertonie haben eine höhere Impfquote, besonders über 60-Jährige.
        Regionale Impfquote: NRW hat die höchste Impfquote unter den Bundesländern.
        Bevölkerungsstruktur: NRW hat eine hohe Bevölkerungsdichte, was die Virusübertragung begünstigt.
        Impfquote nach Alter: Ältere Menschen, besonders über 60, werden häufiger geimpft, was den Schutz der vulnerablen Gruppen verbessert.
        2. Wissenschaftliche Einschätzung der Impfkampagnenwirkung:
        Impfkampagnen führen kurzfristig zu einer Reduktion der Infektionsrate, da mehr Menschen immunisiert werden, was die Übertragung reduziert.
        Herdenimmunitätseffekt: Da die Impfquote unter älteren Menschen hoch ist, wird erwartet, dass schwere Krankheitsverläufe reduziert werden.
        Geschlechtsspezifischer Einfluss: Männer haben eine niedrigere Impfquote, was potenziell zu einer höheren Infektionsrate in dieser Gruppe führen könnte.
        Einfluss der Privatversicherung: Da Männer tendenziell häufiger privat versichert sind, könnte eine unterschiedliche Impfpriorisierung eine Rolle spielen.
        3. Prognose für die Infektionsrate in NRW nach der Impfkampagne:
        Die Impfkampagne wird voraussichtlich die Infektionsrate senken, jedoch nicht sofort drastisch, da Impfungen einige Zeit benötigen, um eine Immunreaktion zu erzeugen. Basierend auf bisherigen Impfkampagnen und der aktuellen Ausgangslage, könnte die Infektionsrate um 10–20% gesenkt werden.

        Geschätzte Infektionsrate nach der Impfkampagne:
        Infektionsrate: [{round(random.uniform(0.1, 2), 2)}]
    """
    return test_text

def lm(prompt):
    openai_api_key = os.getenv("OPENAI_API_KEY")
    # Set your OpenAI API key
    client = OpenAI(api_key=openai_api_key)

    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content

In [95]:
# output = lm("How are you?")
# print(output)

In [96]:
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class CacheEntry(Base):
    __tablename__ = 'cache_entries4'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Region = Column(String)
    Year = Column(Integer)
    Month = Column(Integer)
    InfluenzaCasesPerCapita = Column(Float)
    InfectionRate = Column(Float)
    PreviousInfectionRate = Column(Float)
    Risikogruppe = Column(String)
    Altersgruppe = Column(String)
    Prompt = Column(String)
    Output = Column(String)

# Initialize the database
engine = create_engine('sqlite:///cache.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

class Cache:
    def __init__(self):
        """Initialize the cache with SQLAlchemy session."""
        self.session = Session()

    def add_to_cache(self, region, year, month, influenza_cases_per_capita, infection_rate, prompt, output, risikogruppe, altersgruppe, previous_infection_rate=None):
        """Add or update a row in the cache."""

            # Add new entry
        entry = CacheEntry(
            Region=region,
            Year=int(year) if year else None,
            Month=int(month) if month else None,
            InfluenzaCasesPerCapita=float(influenza_cases_per_capita) if influenza_cases_per_capita else None,
            InfectionRate=float(infection_rate) if infection_rate else None,
            PreviousInfectionRate=float(previous_infection_rate) if previous_infection_rate else None,
            Risikogruppe=risikogruppe,
            Altersgruppe=altersgruppe,
            Prompt=prompt,
            Output=output
        )
        self.session.add(entry)
        self.session.commit()

    def get_from_cache(self, region, year, month):
        """Retrieve a value from the cache."""
        entry = self.session.query(CacheEntry).filter_by(Region=region, Year=year, Month=month).first()
        if entry:
            return {
                'Region': entry.Region,
                'Year': entry.Year,
                'Month': entry.Month,
                'InfluenzaCasesPerCapita': entry.InfluenzaCasesPerCapita,
                'InfectionRate': entry.InfectionRate,
                'PreviousInfectionRate': entry.PreviousInfectionRate,
                'Risikogruppe': entry.Risikogruppe,
                'Altersgruppe': entry.Altersgruppe,
                'Prompt': entry.Prompt,
                'Output': entry.Output
            }
        return None


C:\Users\Amaan\AppData\Local\Temp\ipykernel_56180\3828672870.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [97]:
def extract_between_brackets(text):
    """
    Extracts and returns the content inside square brackets from a string.
    If multiple brackets exist, it returns a list of all matches.

    :param text: The input string.
    :return: A string if one match, a list if multiple matches, or None if no match.
    """
    matches = re.findall(r'\[\[\[(.*?)\]\]\]', text)  # Extract content inside square brackets
    if not matches:
        return None  # Return None if no matches found
    return matches if len(matches) > 1 else matches[0]  # Return list if multiple, string if one


In [98]:
def parse_lm_output(output):
    """
    Parses the output from the language model and extracts the predicted values.

    :param output: The output text from the language model.
    :return: The predicted value(s) as a string or list of strings.
    """

    rate_str = extract_between_brackets(output)
    if rate_str is not None:
        if type(rate_str) != list:
            try:
                return float(rate_str)
            except ValueError:
                return None
        else:
            try:
                return float(rate_str[0])
            except ValueError:
                return None
    return

In [99]:
# Inputs für ein paar der Variablen
# Altergrsuppe: über 60 jährige, unter 60 jährige,
# Risikogruppe: hoher (Risiko), niedriger (Risiko)

def generate_prompt(bundesland, jahr, monat, fallzahlen, infektionsrate, risikogruppe, altersgruppe):
    # Hauptanweisung / Zieldefinition (klar formuliert)
    general_prompt = (
        f"Du bist ein erfahrener Epidemiologe mit fundiertem Wissen über Demografie, "
        f"Gesundheitsinfrastruktur und wissenschaftliche Studien zu Impfungen. "
        f"Deine Aufgabe ist es, die Wirksamkeit einer Impfkampagne in {bundesland} für den "
        f"{monat} {jahr} einzuschätzen, insbesondere hinsichtlich der Reduktion der "
        f"Infektionsrate bei Personen aus der Altersgruppe {altersgruppe} und mit "
        f"{risikogruppe}. Beantworte dazu bitte die folgenden Punkte Schritt für Schritt:\n\n"

        f"1. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate "
        f"   und weiteren relevanten Faktoren.\n"
        f"2. Berücksichtige dein Wissen über die Region (einschließlich Demografie, "
        f"   Gesundheitsinfrastruktur und bisherige Impfergebnisse), um abzuleiten, "
        f"   wie sich diese Faktoren voraussichtlich auf die Infektionsrate auswirken.\n"
        f"3. Erkläre auf Basis wissenschaftlicher Erkenntnisse kurz, wie effektiv die "
        f"   Impfkampagne sein könnte und wodurch sich diese Wirksamkeit begründet.\n"
        f"4. Gib abschließend eine Prognose für die zu erwartende Infektionsrate in "
        f"   {bundesland}, gefolgt von einer kurzen Begründung. Verwende zur Ausgabe deiner "
        f"   endgültigen Schätzung unbedingt das Format: Infektionsrate: [[[Wert]]].\n\n"
    )

    # Datenkontext
    data_knowledge = (f"Relevante Daten aus {bundesland} für {monat} {jahr}: "
                      f"Aktuell sind {round(fallzahlen, 5)}% der Menschen in dieser Gruppe infiziert.\n"
                      )
    if infektionsrate:
        data_knowledge += (
            f"Die Infektionsrate des letzten Monats beträgt {round(infektionsrate, 3)}.\n"
        )
    else:
        data_knowledge += "Aktuell liegen keine konkreten Fallzahlen vor.\n"

    # Weiterführende Hintergrundinformationen (Exploration Knowledge)
    exploration_knowledge = (
        "Zusätzliche Erkenntnisse aus bisherigen Untersuchungen:\n"
        "- Frauen lassen sich deutlich häufiger impfen als Männer.\n"
        "- Männer sind tendenziell etwas öfter privat versichert als Frauen.\n"
        "- Menschen mit Hypertonie werden häufiger geimpft als Menschen mit anderen "
        "  Vorerkrankungen.\n"
        "- Die Impfquote ist in der Altersgruppe über 60 am höchsten.\n"
        "- Bei Risikogruppen: Über-60-Jährige werden exponentiell häufiger geimpft als "
        "  Unter-60-Jährige (außer bei Asthma, wo ein linearer Anstieg vorliegt).\n"
        "- Unter Berücksichtigung der Bevölkerungsdichte ist die Impfquote in den "
        "  verschiedenen Bundesländern sehr ähnlich.\n"
        "- Ebenfalls unter Berücksichtigung der Bevölkerungsdichte nimmt die Impfquote "
        "  linear von 7,8 % auf 4,3 % ab. Die Bundesländer sind absteigend wie folgt "
        "  gerankt: Nordrhein-Westfalen, Niedersachsen, Hessen, Baden-Württemberg, Bayern, "
        "  Bremen, Sachsen-Anhalt, Schleswig-Holstein, Rheinland-Pfalz, Thüringen, Hamburg, "
        "  Brandenburg, Berlin, Mecklenburg-Vorpommern, Sachsen, Saarland.\n"
    )

    return general_prompt + data_knowledge + exploration_knowledge


In [100]:
df = pd.read_csv("final_df.csv", index_col=0, header=0)
df

,Region,Altersgruppe,Inzidenz,Jahr,Woche,risk_groups,Fallzahl,Inzidenz_cumsum,Time,cumsum
10516,Baden-Württemberg,0-29,0.00964,2024,1,Asthma,951,0.00964,2024-01-01,0.00964
10564,Baden-Württemberg,0-29,0.01069,2024,2,Asthma,1055,0.02033,2024-01-08,0.02033
10612,Baden-Württemberg,0-29,0.02613,2024,3,Asthma,2580,0.04646,2024-01-15,0.04646
10660,Baden-Württemberg,0-29,0.04003,2024,4,Asthma,3952,0.08649,2024-01-22,0.08649
10708,Baden-Württemberg,0-29,0.04293,2024,5,Asthma,4239,0.12942,2024-01-29,0.12942
...,...,...,...,...,...,...,...,...,...,...
20839,Thüringen,60-199,0.06115,2025,5,keine,13184,0.42254,2025-01-27,0.15654
20887,Thüringen,60-199,0.04948,2025,6,keine,10667,0.47202,2025-02-03,0.20602
20935,Thüringen,60-199,0.06434,2025,7,keine,13871,0.53636,2025-02-10,0.27036
20983,Thüringen,60-199,0.05558,2025,8,keine,11983,0.59194,2025-02-17,0.32594


In [101]:
df.Altersgruppe.unique()

array(['0-29', '30-59', '60-199'], dtype=object)

In [102]:
df = df.rename(columns={
    "Jahr": "Year",
    "Woche": "CalendarWeek",
    "risk_groups" : "Risikogruppe",
    "Inzidenz" : "Percentage"
})

In [103]:
df.columns

Index(['Region', 'Altersgruppe', 'Percentage', 'Year', 'CalendarWeek',
       'Risikogruppe', 'Fallzahl', 'Inzidenz_cumsum', 'Time', 'cumsum'],
      dtype='object')

In [104]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + df['CalendarWeek'].astype(str) + '1', format='%Y%W%w')
df['Month'] = df['Date'].dt.month
df = df.drop(columns=['Date'])

In [105]:
# existing code...
df.loc[df['Risikogruppe'] != 'keine', 'Risikogruppe'] = 'hohem Risiko'
df.loc[df['Risikogruppe'] == 'keine', 'Risikogruppe'] = 'niedrigem Risiko'
df.loc[df['Altersgruppe'] == '0-29', 'Altersgruppe'] = 'unter 60 jährige'
df.loc[df['Altersgruppe'] == '30-59', 'Altersgruppe'] = 'unter 60 jährige'
df.loc[df['Altersgruppe'] == '60-199', 'Altersgruppe'] = 'über 60 jährige'

In [106]:
df

,Region,Altersgruppe,Percentage,Year,CalendarWeek,Risikogruppe,Fallzahl,Inzidenz_cumsum,Time,cumsum,Month
10516,Baden-Württemberg,unter 60 jährige,0.00964,2024,1,hohem Risiko,951,0.00964,2024-01-01,0.00964,1
10564,Baden-Württemberg,unter 60 jährige,0.01069,2024,2,hohem Risiko,1055,0.02033,2024-01-08,0.02033,1
10612,Baden-Württemberg,unter 60 jährige,0.02613,2024,3,hohem Risiko,2580,0.04646,2024-01-15,0.04646,1
10660,Baden-Württemberg,unter 60 jährige,0.04003,2024,4,hohem Risiko,3952,0.08649,2024-01-22,0.08649,1
10708,Baden-Württemberg,unter 60 jährige,0.04293,2024,5,hohem Risiko,4239,0.12942,2024-01-29,0.12942,1
...,...,...,...,...,...,...,...,...,...,...,...
20839,Thüringen,über 60 jährige,0.06115,2025,5,niedrigem Risiko,13184,0.42254,2025-01-27,0.15654,2
20887,Thüringen,über 60 jährige,0.04948,2025,6,niedrigem Risiko,10667,0.47202,2025-02-03,0.20602,2
20935,Thüringen,über 60 jährige,0.06434,2025,7,niedrigem Risiko,13871,0.53636,2025-02-10,0.27036,2
20983,Thüringen,über 60 jährige,0.05558,2025,8,niedrigem Risiko,11983,0.59194,2025-02-17,0.32594,2


In [107]:
dff = df.groupby(['Year', 'Month', 'Region', 'Risikogruppe', 'Altersgruppe']).sum().reset_index()

In [108]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Percentage,CalendarWeek,Fallzahl,Inzidenz_cumsum,Time,cumsum
0,2024,1,Baden-Württemberg,hohem Risiko,unter 60 jährige,1.44179,210,503480,3.17667,2024-01-012024-01-082024-01-152024-01-222024-0...,3.17667
1,2024,1,Baden-Württemberg,hohem Risiko,über 60 jährige,0.76475,105,246456,1.68399,2024-01-012024-01-082024-01-152024-01-222024-0...,1.68399
2,2024,1,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.20597,30,201682,0.45381,2024-01-012024-01-082024-01-152024-01-222024-0...,0.45381
3,2024,1,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.10925,15,98725,0.24057,2024-01-012024-01-082024-01-152024-01-222024-0...,0.24057
4,2024,1,Bayern,hohem Risiko,unter 60 jährige,2.55136,210,973006,5.42136,2024-01-012024-01-082024-01-152024-01-222024-0...,5.42136
...,...,...,...,...,...,...,...,...,...,...,...
945,2025,3,Schleswig-Holstein,niedrigem Risiko,über 60 jährige,0.04619,9,12069,0.39889,2025-02-24,0.27017
946,2025,3,Thüringen,hohem Risiko,unter 60 jährige,2.02748,126,95810,23.18344,2025-02-242025-02-242025-02-242025-02-242025-0...,10.27432
947,2025,3,Thüringen,hohem Risiko,über 60 jährige,0.41321,63,31772,4.55679,2025-02-242025-02-242025-02-242025-02-242025-0...,2.69479
948,2025,3,Thüringen,niedrigem Risiko,unter 60 jährige,0.28964,18,38380,3.31192,2025-02-242025-02-24,1.46776


In [109]:
cache = Cache()

def send_row_to_model(row):
    previous_record = cache.get_from_cache(row['Region'], row['Year'], row['Month'] - 1)
    previous_rate = previous_record['InfectionRate'] if previous_record else 0

    colname_fallzahlen = 'Percentage'
    prompt = generate_prompt(row['Region'], row['Year'], row['Month'], row[colname_fallzahlen], previous_rate, row['Risikogruppe'], row['Altersgruppe'])
    
    model_output = lm_emulator(prompt)
    parsed_value = parse_lm_output(model_output)
    print("Parsed value:", parsed_value, "  " ,"Model output:", model_output)

    cache.add_to_cache(
        row['Region'], 
        row['Year'], 
        row['Month'], 
        row[colname_fallzahlen],
        parsed_value,  # i.e. infectionrate
        prompt, 
        model_output,
        row['Risikogruppe'],
        row['Altersgruppe'],
        previous_rate
    )
    
    return parsed_value

In [110]:
dff = dff[dff['Region'] == 'Baden-Württemberg']
dff = dff[dff['Month'].isin([7, 8])]
len(dff)

8

In [111]:
dff['predicted_value'] = dff.apply(lambda x: send_row_to_model(x), axis=1)

Du bist ein erfahrener Epidemiologe mit fundiertem Wissen über Demografie, Gesundheitsinfrastruktur und wissenschaftliche Studien zu Impfungen. Deine Aufgabe ist es, die Wirksamkeit einer Impfkampagne in Baden-Württemberg für den 7 2024 einzuschätzen, insbesondere hinsichtlich der Reduktion der Infektionsrate bei Personen aus der Altersgruppe unter 60 jährige und mit hohem Risiko. Beantworte dazu bitte die folgenden Punkte Schritt für Schritt:

1. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate    und weiteren relevanten Faktoren.
2. Berücksichtige dein Wissen über die Region (einschließlich Demografie,    Gesundheitsinfrastruktur und bisherige Impfergebnisse), um abzuleiten,    wie sich diese Faktoren voraussichtlich auf die Infektionsrate auswirken.
3. Erkläre auf Basis wissenschaftlicher Erkenntnisse kurz, wie effektiv die    Impfkampagne sein könnte und wodurch sich diese Wirksamkeit begründet.
4. Gib abschließend eine Prognose für die zu erwartende In

In [112]:
dff = dff[dff['Region'] == 'Baden-Württemberg']
dff.sort_values(by=['Year', "Month"], inplace=True)

In [113]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Percentage,CalendarWeek,Fallzahl,Inzidenz_cumsum,Time,cumsum,predicted_value
382,2024,7,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00448,1624,2119,11.70869,2024-07-012024-07-152024-07-292024-07-012024-0...,0.14392,None
383,2024,7,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00840,1015,2700,8.95769,2024-07-012024-07-082024-07-152024-07-222024-0...,0.10129,None
384,2024,7,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00064,232,855,1.67267,2024-07-012024-07-152024-07-292024-07-012024-0...,0.02056,None
385,2024,7,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00120,145,1085,1.27967,2024-07-012024-07-082024-07-152024-07-222024-0...,0.01447,None
444,2024,8,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00763,1876,3332,12.46742,2024-08-052024-08-122024-08-192024-08-262024-0...,0.18298,None
445,2024,8,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00420,938,1345,7.19117,2024-08-052024-08-122024-08-192024-08-262024-0...,0.10605,None
446,2024,8,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00109,268,1341,1.78106,2024-08-052024-08-122024-08-192024-08-262024-0...,0.02614,None
447,2024,8,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00060,134,542,1.02731,2024-08-052024-08-122024-08-192024-08-26,0.01515,None


In [114]:
dff['previous'] = dff.apply(lambda x: cache.get_from_cache(x['Region'], x['Year'], x['Month'])['PreviousInfectionRate'], axis=1)

In [115]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Percentage,CalendarWeek,Fallzahl,Inzidenz_cumsum,Time,cumsum,predicted_value,previous
382,2024,7,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00448,1624,2119,11.70869,2024-07-012024-07-152024-07-292024-07-012024-0...,0.14392,None,None
383,2024,7,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00840,1015,2700,8.95769,2024-07-012024-07-082024-07-152024-07-222024-0...,0.10129,None,None
384,2024,7,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00064,232,855,1.67267,2024-07-012024-07-152024-07-292024-07-012024-0...,0.02056,None,None
385,2024,7,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00120,145,1085,1.27967,2024-07-012024-07-082024-07-152024-07-222024-0...,0.01447,None,None
444,2024,8,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00763,1876,3332,12.46742,2024-08-052024-08-122024-08-192024-08-262024-0...,0.18298,None,None
445,2024,8,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00420,938,1345,7.19117,2024-08-052024-08-122024-08-192024-08-262024-0...,0.10605,None,None
446,2024,8,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00109,268,1341,1.78106,2024-08-052024-08-122024-08-192024-08-262024-0...,0.02614,None,None
447,2024,8,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00060,134,542,1.02731,2024-08-052024-08-122024-08-192024-08-26,0.01515,None,None
